# Setup

## 참고문서
- https://wikidocs.net/book/14314
- https://wikidocs.net/book/14473

## OpenAI Key 등록
- [OpenAI Key 발급](https://platform.openai.com/api-keys)
- [OpenAI 요금](https://openai.com/api/pricing/)

In [3]:
import os

os.environ['OPENAI_API_KEY'] = '생성한 키 입력'

## 구글 드라이브 연결

In [4]:
# 구글 드라이브 연결(데이터 로드를 위해서)
try:
    from google.colab import drive

    drive.mount('/content/data')
    DATA_PATH = "/content/data/MyDrive/ai_lecture/3. Large Language Models/data/"
except:
    DATA_PATH = "./data/"

Mounted at /content/data


## install

In [3]:
# !pip install --upgrade nltk

In [10]:
!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith langchainhub pymupdf sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 92.8 MB/s eta 0:00:00


# [Retriever](https://wikidocs.net/233779)
- Retrieval Augmented Generation (RAG)에서 `검색도구(Retrievers)` 는 벡터 저장소에서 문서를 검색하는 도구입니다.
- LangChain은 간단한 의미 검색도구부터 성능 향상을 위해 고려된 다양한 검색 알고리즘을 지원합니다.

## Vector Store Retriver
- 벡터스토어 검색도구(Vector Store Retriever)를 사용하면 대량의 텍스트 데이터에서 관련 정보를 효율적으로 검색할 수 있습니다.

### 사전 준비 - 문서 로드 및 분할

In [8]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

loader = PyMuPDFLoader(DATA_PATH+'000660_SK_2023.pdf')
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
    encoding_name='cl100k_base'
)

documents = text_splitter.split_documents(data)
len(documents)

55

### 사전 준비 - 문서 임베딩을 벡터스토어에 저장

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)


vectorstore = FAISS.from_documents(documents,
                                   embedding = embeddings_model,
                                   distance_strategy = DistanceStrategy.COSINE
                                  )

### 단일 문서 검색

In [12]:
query = 'SK주식회사는 4대 핵심 투자 알려줘'

# 가장 유사도가 높은 문장을 하나만 추출
retriever = vectorstore.as_retriever(search_kwargs={'k': 1})

docs = retriever.get_relevant_documents(query)
print(len(docs))
docs[0]

1


<ipython-input-12-f397e0ce9d85>:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  docs = retriever.get_relevant_documents(query)


Document(metadata={'source': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'file_path': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'page': 5, 'total_pages': 21, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 16.2 (Macintosh)', 'producer': 'Adobe PDF Library 15.0', 'creationDate': "D:20230626161631+09'00'", 'modDate': "D:20230626172106+09'00'", 'trapped': ''}, page_content='11\n10\nESG Special Report 2023\nWhere we are heading    |     How we get there    |     What we are preparing\nSK의\n 끊\n임\n없\n는\n 도\n전\n과\n 미\n래\n를\n 앞\n서\n가\n는\n 혜\n안\n을\n \n바\n탕\n으\n로\n SK그\n룹\n의\n 지\n주\n사\n인\n SK주\n식\n회\n사\n는\n, 2021년\n \n투\n자\n전\n문\n회\n사\n로\n의\n 도\n약\n을\n 선\n언\n했\n습\n니\n다\n. SK주\n식\n회\n사\n는\n \n새\n로\n운\n 성\n장\n영\n역\n으\n로\n의\n 포\n트\n폴\n리\n오\n 재\n편\n과\n 더\n불\n어\n, \n기\n업\n가\n치\n 극\n대\n화\n를\n 위\n한\n 또\n 한\n번\n의\n 새\n로\n운\n 성\n장\n 기\n회\n를\n \n모\n색\n하\n고\n 있\n습\n

### MMR(Maximal Marginal Relevance) 검색

In [13]:
# MMR - 다양성 고려 (lambda_mult = 0.5)
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'fetch_k': 50}
)

docs = retriever.get_relevant_documents(query)
print(len(docs))
docs[0]

5


Document(metadata={'source': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'file_path': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'page': 5, 'total_pages': 21, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 16.2 (Macintosh)', 'producer': 'Adobe PDF Library 15.0', 'creationDate': "D:20230626161631+09'00'", 'modDate': "D:20230626172106+09'00'", 'trapped': ''}, page_content='11\n10\nESG Special Report 2023\nWhere we are heading    |     How we get there    |     What we are preparing\nSK의\n 끊\n임\n없\n는\n 도\n전\n과\n 미\n래\n를\n 앞\n서\n가\n는\n 혜\n안\n을\n \n바\n탕\n으\n로\n SK그\n룹\n의\n 지\n주\n사\n인\n SK주\n식\n회\n사\n는\n, 2021년\n \n투\n자\n전\n문\n회\n사\n로\n의\n 도\n약\n을\n 선\n언\n했\n습\n니\n다\n. SK주\n식\n회\n사\n는\n \n새\n로\n운\n 성\n장\n영\n역\n으\n로\n의\n 포\n트\n폴\n리\n오\n 재\n편\n과\n 더\n불\n어\n, \n기\n업\n가\n치\n 극\n대\n화\n를\n 위\n한\n 또\n 한\n번\n의\n 새\n로\n운\n 성\n장\n 기\n회\n를\n \n모\n색\n하\n고\n 있\n습\n

- MMR은 검색 결과의 관련성과 다양성을 균형있게 조정하는 방식입니다.
- `lambda_mult` 매개변수는 관련성과 다양성 사이의 균형을 조정합니다.
- 여기서 `lambda_mult가 0.15`로 설정되어 있으므로, 관련성보다 다양성을 더 우선하게 됩니다.

In [14]:
# MMR - 다양성 고려 (lambda_mult 작을수록 더 다양하게 추출)
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.15}
)

docs = retriever.get_relevant_documents(query)
print(len(docs))
docs[-1]

5


Document(metadata={'source': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'file_path': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'page': 6, 'total_pages': 21, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 16.2 (Macintosh)', 'producer': 'Adobe PDF Library 15.0', 'creationDate': "D:20230626161631+09'00'", 'modDate': "D:20230626172106+09'00'", 'trapped': ''}, page_content='13\n12\nESG Special Report 2023\nWhere we are heading    |     How we get there    |     What we are preparing\n북\n미\n3.5(41%)\n중\n국\n0.8(9%)\nEU\n0.6(7%)\nSK주\n식\n회\n사\n는\n ESG기\n반\n의\n 4대\n 핵\n심\n사\n업\n을\n 중\n심\n으\n로\n \n장\n기\n적\n 관\n점\n의\n 투\n자\n 전\n략\n을\n 실\n행\n하\n고\n 있\n습\n니\n다\n.\n2022년\n에\n는\n 앵\n커\n 자\n산\n과\n 신\n규\n 투\n자\n를\n 연\n계\n한\n 투\n자\n \n성\n과\n로\n 매\n출\n액\n과\n 영\n업\n이\n익\n 성\n장\n을\n 이\n루\n어\n 냈\n습\n니\n다\n. \n또\n한\n 투\n명\n한\n ESG 정\n보\n공\n개\n와\n 이\n해\n관\n계\n자\n 소\n통

### Generation - 답변 생성
-  벡터 저장소에서 문서를 검색한 다음, 이를 기반으로 ChatGPT 모델에 쿼리를 수행하는 end-to-end 프로세스를 구현합니다.
- 이 과정을 통해 사용자의 질문에 대한 의미적으로 관련이 있는 답변을 생성할 수 있습니다.

1. `검색 (Retrieval)`
  - vectorstore.as_retriever를 사용하여 MMR(Maximal Marginal Relevance) 검색 방식으로 문서를 검색합니다.
  - search_kwargs에 k: 5와 lambda_mult: 0.15를 설정하여 상위 5개의 관련성이 높으면서도 다양한 문서를 선택합니다.

In [15]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# Retrieval
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.15}
)

docs = retriever.get_relevant_documents(query)

2. `프롬프트 생성 (Prompt)`
  - ChatPromptTemplate를 사용하여 쿼리에 대한 답변을 생성하기 위한 템플릿을 정의합니다.
  - 여기서 {context}는 검색된 문서의 내용이고, {question}은 사용자의 쿼리입니다.

In [16]:
# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

3. `모델 (Model)`
  - ChatOpenAI를 사용하여 OpenAI의 GPT 모델을 초기화합니다.
  - 이 예에서는 'gpt-3.5-turbo-0125' 모델을 사용하며, temperature를 0으로 설정하여 결정론적인 응답을 생성하고, max_tokens를 500으로 설정하여 응답의 길이를 제한합니다.

In [17]:
# Model
llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0,
    max_tokens=500,
)

4. `문서 포맷팅 (Formatting Docs)`
  - 검색된 문서(docs)를 포맷팅하는 format_docs 함수를 정의합니다.
  - 이 함수는 각 문서의 page_content를 가져와 두 개의 문단 사이에 두 개의 줄바꿈을 삽입하여 문자열로 결합합니다.

In [18]:
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

5. `체인 실행 (Chain Execution)`
  - `prompt | llm | StrOutputParser()`를 사용하여 LLM 체인을 구성하고 실행합니다.
  - 프롬프트를 통해 정의된 쿼리를 모델에 전달하고, 모델의 응답을 문자열로 파싱합니다.

In [19]:
# Chain
chain = prompt | llm | StrOutputParser()

6. `실행 (Run)`
  - chain.invoke 메서드를 사용하여 체인을 실행합니다.
  - context로는 포맷팅된 문서 내용이고, question은 사용자의 쿼리입니다. 최종 응답은 response 변수에 저장됩니다.

In [20]:
# Run
response = chain.invoke({'context': (format_docs(docs)), 'question':query})
response

'SK주식회사는 ESG 기반의 4대 핵심 영역 포트폴리오를 중심으로 장기적 관점의 투자 전략을 실행하고 있습니다. 이 포트폴리오는 첨단소재고성장, 반도체/전력반도체/배터리, 그린 산업의 성장 거점 선점 및 넷제로 실현을 위한 탈탄소 기술 기업 자산 확보, 그리고 디지털 혁신을 선도하는 새로운 주력 사업 영역을 포함하고 있습니다.'

## Multi Query Retriever
- 멀티 쿼리 검색도구(MultiQueryRetriever)는 벡터스토어 검색도구(Vector Store Retriever)의 한계를 극복하기 위해 고안된 방법입니다.
- 사용자가 입력한 쿼리의 의미를 다각도로 포착하여 검색 효율성을 높이고, LLM을 활용하여 사용자에게 보다 관련성 높고 정확한 정보를 제공하는 것을 목표로 합니다.

다음 코드는 MultiQueryRetriever 클래스를 사용하여 여러 쿼리에 기반한 문서 검색 과정을 설정하고 실행하는 방법을 보여줍니다.
1. `MultiQueryRetriever 설정`: from_llm 메서드를 통해, 기존 벡터저장소 검색도구(vectorstore.as_retriever())와 LLM 모델(llm)을 결합하여 MultiQueryRetriever 인스턴스를 생성합니다. 이때 LLM은 다양한 관점의 쿼리를 생성하는 데 사용됩니다.
2. `로깅 설정`: 로깅을 설정하여 MultiQueryRetriever에 의해 생성되고 실행되는 쿼리들에 대한 정보를 로그로 기록하고 확인할 수 있습니다. 검색 과정에서 어떤 쿼리들이 생성되고 사용되었는지 이해하는 데 도움이 됩니다.
3. `문서 검색 실행`: get_relevant_documents 메서드를 사용하여 주어진 사용자 쿼리(question)에 대해 멀티 쿼리 기반의 문서 검색을 실행합니다. 생성된 모든 쿼리에 대해 문서를 검색하고, 중복을 제거하여 고유한 문서들만을 결과로 반환합니다.
4. `결과 확인`: 검색을 통해 반환된 고유 문서들의 수를 확인합니다. 멀티 쿼리 접근 방식을 통해 얼마나 많은 관련 문서가 검색되었는지를 나타냅니다.

In [21]:
from langchain.retrievers.multi_query import MultiQueryRetriever

question = 'SK주식회사는 4대 핵심 투자 알려줘'

llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0,
    max_tokens=500,
)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. SK주식회사의 주요 투자 사업은 무엇인가요?', '2. SK주식회사의 핵심 사업 분야는 무엇인가요?', '3. SK주식회사가 주로 투자하는 사업은 무엇인가요?']


4

In [22]:
unique_docs[1]

Document(metadata={'source': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'file_path': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'page': 5, 'total_pages': 21, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 16.2 (Macintosh)', 'producer': 'Adobe PDF Library 15.0', 'creationDate': "D:20230626161631+09'00'", 'modDate': "D:20230626172106+09'00'", 'trapped': ''}, page_content='식\n회\n사\n, SK C&C 합\n병\n \n및\n 지\n주\n/사\n업\n부\n문\n 구\n분\n•  \x07반\n도\n체\n, 바\n이\n오\n 등\n 유\n망\n 성\n장\n \n전\n망\n 분\n야\n에\n 선\n제\n적\n 투\n자\n2007\n지\n주\n사\n 체\n제\n 전\n환\n• \x07경\n영\n효\n율\n성\n과\n \t\n\t\n재\n무\n건\n전\n성\n 제\n고\n를\n 위\n한\n \t\n선\n진\n 경\n영\n시\n스\n템\n 도\n입\n \nHolding Company  \nOUR JOURNEY\n: Transforming \ninto an Investment \nCompany')

In [23]:
unique_docs[2]

Document(metadata={'source': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'file_path': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'page': 19, 'total_pages': 21, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 16.2 (Macintosh)', 'producer': 'Adobe PDF Library 15.0', 'creationDate': "D:20230626161631+09'00'", 'modDate': "D:20230626172106+09'00'", 'trapped': ''}, page_content='2023 SK Inc. ESG Special Report\n발\n간\n  SK주\n식\n회\n사\n(w\nw\nw\n.sk-inc.com\n)  \n보\n고\n서\n 제\n작\n  Portfolio기\n획\n실\n문\n의\n  서\n울\n 종\n로\n구\n 종\n로\n 26 SK서\n린\n빌\n딩\n  |  sustainability@\nsk.com\n  \n본\n 보\n고\n서\n는\n FSCⓇ\n(Forest Stew\nardship CouncilⓇ\n)인\n증\n을\n 받\n은\n 용\n지\n를\n \n사\n용\n하\n였\n으\n며\n, 콩\n기\n름\n 잉\n크\n로\n 인\n쇄\n되\n었\n습\n니\n다\n. FSCⓇ\n마\n크\n는\n 사\n회\n, 경\n제\n, 환\n경\n적\n \n관\n점\n에\n서\n 지\n속\n가\n능\n하\n게\n 관\n리\n된\n 삼\n림\n내\n에\n서\n 길\n러\n진\n 나\n무\n를\n 사\n용\n한\n 제

앞에서 정의한 MultiQueryRetriever(retriever_from_llm)를 활용하여 여러 쿼리를 생성하고 검색된 문서를 기반으로 사용자 질문에 답변하는 과정을 살펴봅니다.
1. `프롬프트 설정`: 사용자의 질문에 대한 답변을 생성하기 위한 프롬프트 템플릿을 정의합니다. 여기서 {context}는 검색된 문서의 내용을 나타내고, {question}은 사용자의 질문입니다.
2. `모델 초기화`: ChatOpenAI 클래스를 사용하여 GPT-3.5-turbo 모델을 초기화합니다. temperature를 0으로 설정하여 일관된 답변을 생성합니다.
3. `문서 포맷팅 함수 정의`: 검색된 문서들을 하나의 문자열로 포맷팅하는 함수를 정의합니다. 이 함수는 각 문서 내용을 두 개의 줄바꿈으로 구분하여 결합합니다.
4. `체인 정의 및 실행`: RunnablePassthrough를 사용하여 사용자의 질문을 그대로 전달합니다. 이후 검색된 문서(context)와 사용자의 질문(question)을 prompt에 전달하고, 생성된 프롬프트를 GPT-3.5-turbo 모델에 입력으로 제공합니다. StrOutputParser를 사용하여 모델의 출력을 문자열로 파싱합니다.
5. `실행 및 응답 생성`: chain.invoke 메서드를 호출하여 전체 프로세스를 실행합니다. 입력으로 "카카오뱅크의 최근 영업실적을 요약해서 알려주세요."라는 질문을 사용합니다. 이 과정을 통해 검색된 문서의 내용을 기반으로 한 답변이 생성됩니다.

In [24]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# Model
llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0,
)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# Chain
chain = (
    {'context': retriever_from_llm | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Run
response = chain.invoke('SK주식회사는 4대 핵심 투자 알려줘.')
response

INFO:langchain.retrievers.multi_query:Generated queries: ['1. SK주식회사의 주요 투자 분야는 무엇인가요?', '2. SK주식회사가 중점적으로 투자하는 사업은 무엇인가요?', '3. SK주식회사의 주요 투자 사업 부문은 어떤 것이 있나요?']


'SK주식회사는 성장 한계 자산을 선제적, 적극적으로 매각하여 잠재가치를 최대한 회수함으로써 투자의 선순환 체계를 강화하고 있습니다.'

## Contextual compression
- 컨텍스트 압축 기법은 검색된 문서 중에서 쿼리와 관련된 정보만을 추출하여 반환하는 것을 목표로 합니다.
- 쿼리와 무관한 정보를 제거하는 방시으로 답변의 품질을 높이고 비용을 줄일 수 있습니다.

### 기본 검색기 정의

1. `기본 검색기(Base Retriever) 설정`
  - vectorstore.as_retriever 함수를 사용하여 기본 검색기를 설정합니다.
  - 여기서 search_type='mmr'와 search_kwargs={'k':7, 'fetch_k': 20}는 검색 방식을 설정합니다.
  - mmr 검색 방식은 다양성을 고려한 검색 결과를 제공하여, 단순히 가장 관련성 높은 문서만 반환하는 대신 다양한 관점에서 관련된 문서들을 선택합니다.
2. `쿼리 처리 및 문서 검색`
  - base_retriever.get_relevant_documents(question) 함수를 사용하여 주어진 쿼리에 대한 관련 문서를 검색합니다.
  - 이 함수는 쿼리와 관련성 높은 문서들을 반환합니다.
3. `결과 출력`
  - print(len(docs))를 통해 검색된 문서의 수를 출력합니다.

In [25]:
question = 'SK주식회사는 4대 핵심 투자 알려줘.'

llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0,
    max_tokens=500,
)

base_retriever = vectorstore.as_retriever(
                                search_type='mmr',
                                search_kwargs={'k':7, 'fetch_k': 20})

docs = base_retriever.get_relevant_documents(question)
print(len(docs))

7


In [26]:
docs

[Document(metadata={'source': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'file_path': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'page': 5, 'total_pages': 21, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 16.2 (Macintosh)', 'producer': 'Adobe PDF Library 15.0', 'creationDate': "D:20230626161631+09'00'", 'modDate': "D:20230626172106+09'00'", 'trapped': ''}, page_content='11\n10\nESG Special Report 2023\nWhere we are heading    |     How we get there    |     What we are preparing\nSK의\n 끊\n임\n없\n는\n 도\n전\n과\n 미\n래\n를\n 앞\n서\n가\n는\n 혜\n안\n을\n \n바\n탕\n으\n로\n SK그\n룹\n의\n 지\n주\n사\n인\n SK주\n식\n회\n사\n는\n, 2021년\n \n투\n자\n전\n문\n회\n사\n로\n의\n 도\n약\n을\n 선\n언\n했\n습\n니\n다\n. SK주\n식\n회\n사\n는\n \n새\n로\n운\n 성\n장\n영\n역\n으\n로\n의\n 포\n트\n폴\n리\n오\n 재\n편\n과\n 더\n불\n어\n, \n기\n업\n가\n치\n 극\n대\n화\n를\n 위\n한\n 또\n 한\n번\n의\n 새\n로\n운\n 성\n장\n 기\n회\n를\n \n모\n색\n하\n고\n 있\n습\

### 문서 압축기의 구성과 작동 방식
- 문서 압축기는 기본 검색기로부터 얻은 문서들을 더욱 효율적으로 압축하여, 쿼리와 가장 관련이 깊은 내용만을 추려내는 것을 목표로 합니다.
- `LLMChainExtractor`와 `ContextualCompressionRetriever` 클래스를 사용합니다.

1. `LLMChainExtractor 설정`
  - LLMChainExtractor.from_llm(llm)를 사용하여 문서 압축기를 설정합니다.
  - 언어 모델(llm)을 사용하여 문서 내용을 압축합니다.
2. `ContextualCompressionRetriever 설정`
  - ContextualCompressionRetriever 인스턴스를 생성할 때, base_compressor와 base_retriever를 인자로 제공합니다.
  - base_compressor는 앞서 설정한 LLMChainExtractor 인스턴스이며, base_retriever는 기본 검색기 인스턴스입니다.
  - 이 두 구성 요소를 결합하여 검색된 문서들을 압축하는 과정을 처리합니다.
3. `압축된 문서 검색`
  - compression_retriever.get_relevant_documents(question) 함수를 사용하여 주어진 쿼리에 대한 압축된 문서들을 검색합니다.
  - 기본 검색기를 통해 얻은 문서들을 문서 압축기를 사용하여 내용을 압축하고, 쿼리와 가장 관련된 내용만을 추려냅니다.
4. `결과 출력`
  - print(len(compressed_docs))를 통해 압축된 문서의 수를 출력합니다.

In [27]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=base_retriever
)

compressed_docs = compression_retriever.get_relevant_documents(question)
print(len(compressed_docs))

4


In [28]:
compressed_docs

[Document(metadata={'source': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'file_path': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'page': 5, 'total_pages': 21, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 16.2 (Macintosh)', 'producer': 'Adobe PDF Library 15.0', 'creationDate': "D:20230626161631+09'00'", 'modDate': "D:20230626172106+09'00'", 'trapped': ''}, page_content='SK주식회사는 4대 핵심 투자 중심의 성장 방식을 구체화하였습니다.'),
 Document(metadata={'source': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'file_path': '/content/data/MyDrive/ai_lecture/3. Large Language Models/data/000660_SK_2023.pdf', 'page': 6, 'total_pages': 21, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 16.2 (Macintosh)', 'producer': 'Adobe PDF Library 15.0', 'creationDate': "D:20230626161631+09

In [29]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# Model
llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0,
)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# Chain
chain = (
    {'context': compression_retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Run
response = chain.invoke('SK주식회사는 4대 핵심 투자 알려줘.')
response

'SK주식회사의 4대 핵심 투자 영역은 첨단소재고성장, GREEN 산업, 디지털 혁신, 그리고 후보군 발굴 및 신기술 투자입니다.'

# [Reranker](https://wikidocs.net/253434)

# [Agent](https://wikidocs.net/233782)

# [LangGraph](https://wikidocs.net/233785)